## Обработка видеофайла

In [7]:
import cv2, time, datetime
from numba import jit

# Параметры настройки
t0 = 1672531200    # указать (получить) время начала создания видеофайла прямого контроля, с
fps = 30.04         # число кадров в секунду
d = 0.2             # задержка между кадрами в паре, с
n = 0.1             # относительный размер ячеек
m = 0.2             # относительный шаг контрольных точек в ячейках
sx = 0.25           # шаг смещения ячеек по горизонтали
sy = 1              # шаг смещения ячеек по вертикали
k = 500             # коэффициент отсечки по диф. сумме ячейки
dt_start = 1        # задержка перед началом движения в кадре, с
dt_end = 3          # задержка после окончания движения в кадре, с


# вычисление дифференциальных сумм для ячеек кадров
#@jit
def cell_sums(img_1, img_2, n, m, sx, sy, k):
    diffsums = []
    l_y = int(len(img_1)*n)         # размер ячеек по вертикали
    l_x = int(len(img_1[0])*n)      # размер ячеек по горизонтали    
    for i in range(0, len(img_1)-l_y, int(l_y*sy)):           
        for j in range(0, len(img_1[0])-l_x, int(l_x*sx)):             
            cell_sums_1, cell_sums_2 = [0, 0, 0], [0, 0, 0]
            for ii in range(i, i+l_y, int(l_y*m)):     # l_y*m - шаг сетки контрольных точек по вертикали
                for jj in range(j, j+l_x, int(l_x*m)): # l_x*m - шаг сетки контрольных точек по горизонтали           
                    cell_sums_1[0] += int(img_1[ii][jj][0])
                    cell_sums_2[0] += int(img_2[ii][jj][0])
                    cell_sums_1[1] += int(img_1[ii][jj][1])
                    cell_sums_2[1] += int(img_2[ii][jj][1])
                    cell_sums_1[2] += int(img_1[ii][jj][2])
                    cell_sums_2[2] += int(img_2[ii][jj][2])

            s = abs(cell_sums_2[0] - cell_sums_1[0]) + abs(cell_sums_2[1] - cell_sums_1[1]) + abs(cell_sums_2[2] - cell_sums_1[2])
            if s > k: diffsums.append(s)     # в список диф. сумм добавляются только диф. суммы ячеек, большие k 
    return sum(diffsums) 


dir_name = './test_video/детекция движения/307_test/из файла'
cap = cv2.VideoCapture(f'{dir_name}/307.mp4') 
t1 = time.time()

reg = False
sum_video = 0
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))         # число кадров в видео-файле
T = int(frame_count/fps)                                     # длительность видео-файла 
st = int(fps*d)                                              # шаг по кадрам
num = 1                                                      # номер видеофрагмента с движением 
mov_data = dict()                                            # словарь с данными о движении 

for i in range(0, frame_count - st, st):                     # обработка кадров с заданным шагом
    cap.set(1, i)                                            # (1, номер кадра - 1)
    t = round(i/fps, 3)                                      # время кадра в файле
    ret, frame_1 = cap.read()
    cap.set(1, i + st)                                       # задержка между кадрами
    ret, frame_2 = cap.read()
    diffsum = cell_sums(frame_1, frame_2, n, m, sx, sy, k)
    
    if reg == False:
        if diffsum == 0: continue  # пропуск кадров
        else: 
            reg = True
            t_start = t
            t_reg = t
            sum_video = diffsum
    else:
        if diffsum > 0 and i < frame_count - 2*st:
            sum_video += diffsum 
            t_reg = t
        elif diffsum == 0 and t - t_reg < dt_end and i < frame_count - 2*st: 
            continue
        else:
            reg = False          
            t_end = t
            if t - t_reg < dt_end:
                dt_end = t - t_reg
            dt = round(t_end - t_start - dt_end, 1)      # длительность движения в видеофрагменте
            # время начала видеофрагмента с движением
            DT1 = datetime.datetime.fromtimestamp(t0 + max(t_start-dt_start, 0)).strftime('%Y-%m-%d_%H-%M-%S')  
            # время окончания видеофрагмента с движением
            DT2 = datetime.datetime.fromtimestamp(t0 + int(t_end)).strftime('%Y-%m-%d_%H-%M-%S')  
            ai = round(sum_video/dt)                     # средняя интенсивность движения во время ролика            
                      
            mov_data[num] = (DT1, DT2, dt, ai) # запись данных о движении в словарь
            sum_video = 0
            num += 1
    
cap.release()
t2 = time.time()

print('длительность видео-файла, m-s:', str(T//60) + '-' + str(T%60))
print('длительность обработки, m-s:', str(round(t2 - t1)//60) + '-' + str(round(t2 - t1)%60))
print(mov_data)

длительность видео-файла, m-s: 1-29
длительность обработки, m-s: 0-56
{1: ('2023-01-01_03-00-14', '2023-01-01_03-00-26', 8.6, 197249), 2: ('2023-01-01_03-00-30', '2023-01-01_03-00-40', 5.6, 22734), 3: ('2023-01-01_03-00-39', '2023-01-01_03-01-05', 21.2, 300973), 4: ('2023-01-01_03-01-13', '2023-01-01_03-01-22', 4.8, 140790), 5: ('2023-01-01_03-01-23', '2023-01-01_03-01-28', 2.0, 6933)}


## Обработка аудиофайла

In [2]:
import time, datetime, os
from scipy import signal
from scipy.io import wavfile

# Параметры настройки
t0 = 1672531200      # указать (получить) время начала создания аудиофайла, с
d = 0.1               # задержка между спектрами в паре, с
n = 0.01              # относительная ширина интервалов частот
k = 15                # коэффициент отсечки по диф. сумме интервала частот
dt_start = 1          # задержка перед началом началом изменения звука, с
dt_end = 2            # задержка после окончания изменения звука, с

# вычисление дифференциальных сумм для интервалов частот
def band_sums(sp_1, sp_2, n, k):
    diffsums = []
    l = int(len(sp_1)*n)  # ширина интервала частот
    for i in range(0, len(sp_1) - l, l):
        band_sum_1, band_sum_2 = 0, 0
        for j in range(i, i+l):
            band_sum_1 += int(sp_1[j])
            band_sum_2 += int(sp_2[j])
        s = abs(band_sum_2 - band_sum_1)
        if s > k: diffsums.append(s)
    return sum(diffsums)

t1 = time.time()

dir_name = './test_audio/детекция изменений/из файла'
sample_rate, samples = wavfile.read(f'{dir_name}/307_mono.wav')
frequencies, times, spectrogram = signal.spectrogram(samples, sample_rate)

reg = False
sum_audio = 0
dt_min = times[1]-times[0]
T = round(times[-1])
st = int(d/dt_min)   # шаг по времени
num = 1
mov_audio_data = dict()  

for i in range(0, len(spectrogram[0])-st, st):
    sp_1, sp_2 = [], []
    t = round(i*dt_min, 3)
    for j in range(len(spectrogram)):
        sp_1.append(spectrogram[j][i])
        sp_2.append(spectrogram[j][i+st])
    diffsum = band_sums(sp_1, sp_2, n, k)
    
    if reg == False:
        if diffsum == 0: continue  # пропуск спектров
        else: 
            reg = True
            t_start = t
            t_reg = t
            sum_audio = diffsum
    else:
        if diffsum > 0 and i < len(spectrogram[0]) - 2*st:
            sum_audio += diffsum 
            t_reg = t
        elif diffsum == 0 and t - t_reg < dt_end and i < len(spectrogram[0]) - 2*st: 
            continue
        else:
            reg = False          
            t_end = t
            if t - t_reg < dt_end:
                dt_end = t - t_reg
            dt = round(t_end - t_start - dt_end, 1)   # длительность звуковых изменений в аудиофрагменте
            # время начала видеофрагмента с движением
            DT1 = datetime.datetime.fromtimestamp(t0 + max(t_start-dt_start, 0)).strftime('%Y-%m-%d_%H-%M-%S')  
            # время окончания видеофрагмента с движением
            DT2 = datetime.datetime.fromtimestamp(t0 + int(t_end)).strftime('%Y-%m-%d_%H-%M-%S')  
            ai = round(sum_audio/dt)    # средняя интенсивность звуковых изменений за время аудиофрагмента           
            
            mov_audio_data[num] = (DT1, DT2, dt, ai) # запись данных о звуковых изменениях в словарь
            sum_audio = 0
            num += 1
    

t2 = time.time()

print('длительность аудиоролика, m-s:', str(T//60) + '-' + str(T%60))
print('длительность обработки, m-s:', str(round(t2 - t1)//60) + '-' + str(round((t2 - t1)%60, 2)))
print(mov_audio_data)

длительность аудиоролика, m-s: 1-11
длительность обработки, m-s: 0-0.2
{1: ('2022-09-23_16-44-39', '2022-09-23_16-44-44', 0.3, 6123), 2: ('2022-09-23_16-44-43', '2022-09-23_16-44-48', 0.1, 160), 3: ('2022-09-23_16-44-47', '2022-09-23_16-44-52', 0.3, 250), 4: ('2022-09-23_16-44-51', '2022-09-23_16-44-57', 0.2, 400), 5: ('2022-09-23_16-44-56', '2022-09-23_16-45-02', 0.4, 11265), 6: ('2022-09-23_16-45-02', '2022-09-23_16-45-07', 0.2, 3525), 7: ('2022-09-23_16-45-07', '2022-09-23_16-45-13', 0.2, 2725), 8: ('2022-09-23_16-45-16', '2022-09-23_16-45-23', 2.1, 47), 9: ('2022-09-23_16-45-29', '2022-09-23_16-45-34', 0.2, 460)}
